In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
#The purpose of this book is to compare in one plot the results of varying parameters.

FM2xy_camera_1000ms_40x = np.genfromtxt('./FM2xy_camera_1000ms_40x.csv',delimiter =",")
FM2xy_camera_1000ms_60x = np.genfromtxt('./FM2xy_camera_1000ms_60x.csv',delimiter =",")
FM2xy_camera_625ms_40x = np.genfromtxt('./FM2xy_camera_625ms_40x.csv',delimiter =",")
FM2xy_camera_625ms_60x = np.genfromtxt('./FM2xy_camera_625ms_60x.csv',delimiter =",")

FM2xy_confocal_1000ms_40x = np.genfromtxt('./FM2xy_confocal_1000ms_40x.csv',delimiter =",")
FM2xy_confocal_1000ms_60x = np.genfromtxt('./FM2xy_confocal_1000ms_60x.csv',delimiter =",")
FM2xy_confocal_625ms_40x = np.genfromtxt('./FM2xy_confocal_625ms_40x.csv',delimiter =",")
FM2xy_confocal_625ms_60x = np.genfromtxt('./FM2xy_confocal_625ms_60x.csv',delimiter =",")[0:-1]

In [3]:
# geoM2xy_camera_1000ms_60x = np.genfromtxt('./geoM2xy_camera_1000ms_60x.csv',delimiter =",")
# arM2xy_camera_1000ms_60x = np.genfromtxt('./arM2xy_camera_1000ms_60x.csv',delimiter =",")

In [4]:
f1 = 1.62
f2 = 1.02

time1 = np.zeros(FM2xy_confocal_625ms_60x.shape[0])
time2 = np.zeros(FM2xy_confocal_1000ms_60x.shape[0])

for num in range(0, FM2xy_confocal_625ms_60x.shape[0]):
    time1[num] = num/f1

for num in range(0, FM2xy_confocal_1000ms_60x.shape[0]):
    time2[num] = num/f2

In [5]:
#Linear algebra to find Deff (1000ms):
to_frame1 = 48

w = dict()
line = dict()
A1 = np.ones((np.shape(time2)[0], 2))
A1[:, 0] = time2

w[0] = np.linalg.lstsq(A1[1:to_frame1, :], FM2xy_camera_1000ms_40x[1:to_frame1])[0]
w[1] = np.linalg.lstsq(A1[1:to_frame1, :], FM2xy_camera_1000ms_60x[1:to_frame1])[0]
w[2] = np.linalg.lstsq(A1[1:to_frame1, :], FM2xy_confocal_1000ms_40x[1:to_frame1])[0]
w[3] = np.linalg.lstsq(A1[1:to_frame1, :], FM2xy_confocal_1000ms_60x[1:to_frame1])[0]

line[0] = w[0][0]*time2 + w[0][1]
line[1] = w[1][0]*time2 + w[1][1]
line[2] = w[2][0]*time2 + w[2][1]
line[3] = w[3][0]*time2 + w[3][1]

In [6]:
print('Diffusion Coeff (um^2/s):', np.round(w[0][0], 7), 'FM2xy_camera_1000ms_40x')
print('Diffusion Coeff (um^2/s):', np.round(w[1][0], 7), 'FM2xy_camera_1000ms_60x')
print('Diffusion Coeff (um^2/s):', np.round(w[2][0], 7), 'FM2xy_confocal_1000ms_40x')
print('Diffusion Coeff (um^2/s):', np.round(w[3][0], 7), 'FM2xy_confocal_1000ms_60x')

Diffusion Coeff (um^2/s): 4e-06 FM2xy_camera_1000ms_40x
Diffusion Coeff (um^2/s): 0.0001785 FM2xy_camera_1000ms_60x
Diffusion Coeff (um^2/s): 0.0001317 FM2xy_confocal_1000ms_40x
Diffusion Coeff (um^2/s): 9.76e-05 FM2xy_confocal_1000ms_60x


In [7]:
#Linear algebra to find Deff (1000ms):
to_frame = 67

w1 = dict()
line1 = dict()
A2 = np.ones((np.shape(time1)[0], 2))
A2[:, 0] = time1


w1[0] = np.linalg.lstsq(A2[1:to_frame, :], FM2xy_camera_625ms_40x[1:to_frame])[0]
w1[1] = np.linalg.lstsq(A2[1:to_frame, :], FM2xy_camera_625ms_60x[1:to_frame])[0]
w1[2] = np.linalg.lstsq(A2[1:to_frame, :], FM2xy_confocal_625ms_40x[1:to_frame])[0]
w1[3] = np.linalg.lstsq(A2[1:to_frame, :], FM2xy_confocal_625ms_60x[1:to_frame])[0]

line1[0] = w1[0][0]*time1 + w1[0][1]
line1[1] = w1[1][0]*time1 + w1[1][1]
line1[2] = w1[2][0]*time1 + w1[2][1]
line1[3] = w1[3][0]*time1 + w1[3][1]

In [8]:
print('Diffusion Coeff (um^2/s):', np.round(w1[0][0], 7), 'FM2xy_camera_625ms_40x')
print('Diffusion Coeff (um^2/s):', np.round(w1[1][0], 7), 'FM2xy_camera_625ms_60x')
print('Diffusion Coeff (um^2/s):', np.round(w1[2][0], 7), 'FM2xy_confocal_625ms_40x')
print('Diffusion Coeff (um^2/s):', np.round(w1[3][0], 7), 'FM2xy_confocal_625ms_60x')

Diffusion Coeff (um^2/s): 4.28e-05 FM2xy_camera_625ms_40x
Diffusion Coeff (um^2/s): 0.0001556 FM2xy_camera_625ms_60x
Diffusion Coeff (um^2/s): 0.0010982 FM2xy_confocal_625ms_40x
Diffusion Coeff (um^2/s): 0.0005591 FM2xy_confocal_625ms_60x


In [41]:
fig = plt.figure()
ax = fig.add_subplot(111)

## the data
N = 4

## necessary variables
ind = np.arange(N)                # the x locations for the groups
width = 0.6                     # the width of the bars

## the bars
rects1 = ax.bar(ind, y1, width,
                color='green')

# axes and labels
ax.set_xlim(-width/2,len(ind)+width/2)
ax.set_ylim(0,0.0012)
ax.set_ylabel('Diffusion Coefficient (um^2/s)')
ax.set_title('Diffusion coefficients for camera')
xTickMarks = [labels1[i-1] for i in range(1,5)]
ax.set_xticks(ind+width)
xtickNames = ax.set_xticklabels(xTickMarks)
plt.setp(xtickNames, rotation=45, fontsize=10)

plt.show()

In [42]:
fig = plt.figure()
ax = fig.add_subplot(111)

## the data
N = 4

## necessary variables
ind = np.arange(N)                # the x locations for the groups
width = 0.6                     # the width of the bars

## the bars
rects1 = ax.bar(ind, y2, width,
                color='green')

# axes and labels
ax.set_xlim(-width/2,len(ind)+width/2)
ax.set_ylim(0,0.0012)
ax.set_ylabel('Diffusion Coefficient (um^2/s)')
ax.set_title('Diffusion coefficients for confocal')
xTickMarks = [labels2[i-1] for i in range(1,5)]
ax.set_xticks(ind+width)
xtickNames = ax.set_xticklabels(xTickMarks)
plt.setp(xtickNames, rotation=45, fontsize=10)

plt.show()

In [13]:
N = 5

ind = np.arange(N)
width = 0.35     

x1 = [1, 2, 3, 4]
x2 = [1, 2, 3, 4]
y1 = [w[0][0], w[1][0], w1[0][0], w1[1][0]]
y2 = [w[2][0], w[3][0], w1[2][0], w1[3][0]]

labels1 = ['FM2xy_camera_1000ms_40x', 'FM2xy_camera_1000ms_60x', 'FM2xy_camera_625ms_40x', 'FM2xy_camera_625ms_60x']
labels2 = ['FM2xy_confocal_1000ms_40x', 'FM2xy_confocal_1000ms_60x', 'FM2xy_confocal_625ms_40x', 'FM2xy_confocal_625ms_60x']

In [ ]:
FM2xy_confocal_625ms_60x

In [ ]:
time2

In [ ]:
#Figure parameters

filename = 'FBF_comparison_40x.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
ax.plot(time1, FM2xy_camera_625ms_40x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 40x')
ax.plot(time2, FM2xy_confocal_1000ms_40x, 'go', ms=mi, label='Confocal 1000ms 40x')
ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 40x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
time1.shape

In [ ]:
#Figure parameters

filename = 'FBF_comparison_60x.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_camera_1000ms_60x, 'bo', ms=mi, label='Camera 1000ms 60x')
ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
ax.plot(time2, FM2xy_confocal_1000ms_60x, 'go', ms=mi, label='Confocal 1000ms 60x')
ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'FBF_comparison_625ms.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time1, FM2xy_camera_625ms_40x, 'bo', ms=mi, label='Camera 625ms 40x')
ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'go', ms=mi, label='Confocal 625ms 40x')
ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'FBF_comparison_1000ms.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
ax.plot(time2, FM2xy_confocal_1000ms_40x, 'go', ms=mi, label='Confocal 1000ms 40x')
ax.plot(time2, FM2xy_confocal_1000ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'FBF_comparison_camera.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
ax.plot(time1, FM2xy_camera_625ms_40x, 'go', ms=mi, label='Camera 625ms 40x')
ax.plot(time1, FM2xy_camera_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'FBF_comparison_confocal.png'

limit1=50.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.051 #y range
tick2 = round(limit2, 2)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=2 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2, FM2xy_confocal_1000ms_40x, 'bo', ms=mi, label='Confocal 1000ms 40x')
ax.plot(time2, FM2xy_confocal_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'go', ms=mi, label='Confocal 625ms 40x')
ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

ax.plot(time2, line[2], linewidth=3, color='purple')
ax.plot(time2, line[3], linewidth=3, color='purple')
ax.plot(time1, line1[2], linewidth=3, color='purple')
ax.plot(time1, line1[3], linewidth=3, color='purple')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'FBF_geo_ar_comparison.png'

limit1=20.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=0.0051 #y range
tick2 = round(limit2, 3)/5 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=3 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time2[0:19], geoM2xy_camera_1000ms_60x[0:-1], 'b--', linewidth=wi, dashes=[d1,d2], label='Geometric')
ax.plot(time2[0:19], arM2xy_camera_1000ms_60x[0:-1], 'g--', linewidth=wi, dashes=[d1,d2], label='Arithmetic')
ax.plot(time2, FM2xy_camera_1000ms_60x, 'r--', linewidth=wi, dashes=[d1,d2], label='Frame-by-frame')
    
# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(t, line[0], linewidth=3, color='blue')
# ax.plot(t, line[1], linewidth=3, color='red')
# ax.plot(t, line[2], linewidth=3, color='green')
    
#ax.plot(t, linel[0], linewidth=3, color='blue')
#ax.plot(t, linel[1], linewidth=3, color='red')
#ax.plot(t, linel[2], linewidth=3, color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.06, 0.66), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0, limit2])

# Save your figure
plt.savefig('{}.png'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
geoM2xy_camera_1000ms_60x.shape